In [3]:
import os

In [1]:
%pwd

'd:\\MLOps Projects\\StellarClassifierOps\\research'

In [4]:
os.chdir("../")

In [20]:
%pwd

'd:\\MLOps Projects\\StellarClassifierOps'

In [41]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    test_size: float
    random_state: int
    target_column: str
    numerical_columns: list

In [42]:
from src.StellarClassifier.constants import *
from src.StellarClassifier.utils.common import read_yaml, create_directories

In [43]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath= CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        params = self.params.data_transformation
        schema = self.schema
        create_directories([config.root_dir])
        data_transformation_config=DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            test_size= params.test_size,
            random_state=params.random_state,
            target_column= schema.TARGET_COLUMN['name'],
            numerical_columns= [col for col, dtype in schema['COLUMNS'].items() if dtype == 'float64']
        )
        return data_transformation_config

In [44]:
import os
from src.StellarClassifier import logger
from sklearn.model_selection import train_test_split
import pandas as pd


from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE

In [49]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    
    def train_test_spliting(self):
        # Load data
        data = pd.read_csv(self.config.data_path)
        logger.info("Loaded data for train-test split.")

        # Split data
        train, test = train_test_split(
            data,
            test_size=self.config.test_size,
            random_state=self.config.random_state
        )
        logger.info("Split data into training and test sets.")
        return train, test

    def apply_smote(self, train_data):
        logger.info("Applying SMOTE for class imbalance...")
        smote = SMOTE(random_state=self.config.random_state)
        X = train_data.drop(columns=[self.config.target_column])
        y = train_data[self.config.target_column]

        #shape before SMOTE
        logger.info(f"shape before SMOTE: Features: {X.shape}, Target: {y.shape}")

        X_resampled, y_resampled = smote.fit_resample(X, y)

        logger.info(f"Shape after SMOTE: Features: {X_resampled.shape}, Target: {y_resampled.shape}")

        resampled_data = pd.concat([pd.DataFrame(X_resampled), pd.Series(y_resampled)], axis=1)
        logger.info("SMOTE applied successfully.")
        return resampled_data

    def normalize_data(self, data):
        logger.info("Normalizing data...")
        scaler = MinMaxScaler()

        # Normalize only numerical columns
        numerical_data = data[self.config.numerical_columns]
        scaled_values = scaler.fit_transform(numerical_data)
        
        # Replace original columns with normalized values
        data[self.config.numerical_columns] = scaled_values
        logger.info("Data normalization completed.")
        return data


In [50]:
try:
    config = ConfigurationManager()
    data_trasformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_trasformation_config)

    # Train-test split
    train, test = data_transformation.train_test_spliting()

    # Apply SMOTE
    resampled_train = data_transformation.apply_smote(train)

    # Normalize data
    normalized_train = data_transformation.normalize_data(resampled_train)
    normalized_test = data_transformation.normalize_data(test)

    # Save processed data
    normalized_train.to_csv(os.path.join(config.get_data_transformation_config().root_dir, "normalized_train.csv"), index=False)
    normalized_test.to_csv(os.path.join(config.get_data_transformation_config().root_dir, "normalized_test.csv"), index=False)

    logger.info("Data Transformation pipeline completed.")

except Exception as e: 
    raise e

[2024-12-13 09:28:31,980: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-13 09:28:31,982: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-13 09:28:31,983: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-13 09:28:31,984: INFO: common: created directory at: artifacts]
[2024-12-13 09:28:31,984: INFO: common: created directory at: artifacts/data_transformation]
[2024-12-13 09:28:32,053: INFO: 1008500690: Loaded data for train-test split.]
[2024-12-13 09:28:32,061: INFO: 1008500690: Split data into training and test sets.]
[2024-12-13 09:28:32,063: INFO: 1008500690: Applying SMOTE for class imbalance...]
[2024-12-13 09:28:32,064: INFO: 1008500690: shape before SMOTE: Features: (74999, 6), Target: (74999,)]
[2024-12-13 09:28:32,386: INFO: 1008500690: Shape after SMOTE: Features: (133704, 6), Target: (133704,)]
[2024-12-13 09:28:32,392: INFO: 1008500690: SMOTE applied successfully.]
[2024-12-13 09:28:32,394: INFO: 1008500690: No